<font face="Calibri">
<br>
<font size="6"> <b> AVO Obsarviewory:<b> </font>
<font size="5"> <b> routine InSAR processing tool for Alaska volcanoes<b> </font></b>
<br>
<br>
<font size="5"> <b> Current Notebook: select_processing<b> </font></b>

<br>
<br>
<br>
<font size="4"> 
This notebook runs a MintPy analysis for a selected volcano.
<br>
<br>
This Notebook is part of <a href="https://github.com/uafgeotools/avo_obsarviewtory" target="_blank">AVO Obsarviewtory</a>.  


</font>
<hr>

In [2]:
import pickle
class volcano_area: # see avoobs_init for details
    def __init__(self, volc_name, filter_dates, ul, lr, path, frame, asf_name, run_flag):
        self.volc_name = volc_name
        self.filter_dates = filter_dates
        self.ul = ul
        self.lr = lr
        self.path = path
        self.frame = frame
        self.asf_name = asf_name
        self.run_flag = run_flag
    def update_flag(self, new_flag):
        self.run_flag = new_flag

with open('data/volclist.pkl', 'rb') as f: # this file contains volcano information
    volcano_list = pickle.load(f)

projects = [] # create a list to contain project names
for volcano in volcano_list:
    projects.append(volcano.volc_name+str(volcano.path)) # this results in project names like "seguam15" or "akutan44"
import ipywidgets as widgets
print('Choose a project')
project_selector   = widgets.RadioButtons(
    options     = list(projects),
    description = 'project:',
    disabled=False,
)
display(project_selector)

Choose a project


RadioButtons(description='project:', options=('seguam15', 'chagulak15', 'yunaska15', 'kiska30', 'davidof30', '…

In [ ]:
import avo_insar_functions
import hyp3_sdk as sdk
from pathlib import Path

print('Current processing: '+project_selector.value)
volc_index = projects.index(project_selector.value) # collect the index so we know which volcano is selected 
class asf_area:
    def __init__(self, asf_name, path, frame, last_date): 
        self.asf_name = asf_name
        self.path = path
        self.frame = frame
        self.last_date = last_date
    def update_date(self,new_date):
        self.last_date = new_date
with open('data/asflist.pkl', 'rb') as f: 
    asf_list = pickle.load(f)


asf_names = []
for asf_project in asf_list: 
    asf_names.append(asf_project.asf_name)
    

volcano = volcano_list[volc_index] # use the index to locate the volcano that is selected
date_index = asf_names.index(volcano.asf_name) # find the index for the same volcano from asf projects


hyp3 = sdk.HyP3('https://hyp3-avo.asf.alaska.edu', prompt=True, username='ycheng', password='earthdata+1S')
update_flag = avo_insar_functions.update_asf_project(volcano.path,volcano.frame,volcano.asf_name,asf_list[date_index].last_date,hyp3)

if update_flag == 1:
    jobs = hyp3.find_jobs(name=volcano.asf_name)
    jobs = hyp3.watch(jobs) # wait for complete

    print('########CURRENT PROJECT########')
    project_name =volcano.volc_name + str(volcano.path) # this results in project names like "seguam15" or "akutan44"
    print(project_name)
    
    analysis_directory = Path.cwd() / 'processings' / project_name
    mintpy_directory = analysis_directory / 'MintPy'
    print(f"analysis_directory: {analysis_directory}")

    gdal_command = avo_insar_functions.avo_insar_download( hyp3 , volcano.asf_name , analysis_directory , volcano.filter_dates) # download
    full_scene = analysis_directory/"full_scene.tif"
    if full_scene.exists():
        full_scene.unlink()
    !{gdal_command}
    image_file = f"{analysis_directory}/raster_stack.vrt"
    !gdalbuildvrt -separate $image_file -overwrite $full_scene
    avo_insar_functions.avo_insar_crop(image_file, volcano.ul, volcano.lr, analysis_directory) # crop downloaded files into pre-defined area of interest

    mintpy_config = avo_insar_functions.avo_insar_run( analysis_directory , mintpy_directory ) # run MintPy
    !smallbaselineApp.py --dir {mintpy_directory} {mintpy_config}

    from mintpy import view
    view.main([f'{mintpy_directory}/velocity.h5']) # check products